In [1]:
%matplotlib qt

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import dataloader
from sklearn.model_selection import train_test_split
from preprocessing import Normalize_df, WindSpeedDataset, ComposeTransform, ToTensor

In [2]:
dataset = Normalize_df(pd.read_csv('./dataset-daily.csv'))
dataset.head()

,time,air_temperature_mean,pressure,wind_direction,wind_speed
0,0.000000,0.370203,0.103164,0.732591,0.625000
1,0.000011,0.322799,0.268912,0.838440,0.354167
2,0.000022,0.302483,0.709078,0.988858,0.260417
3,0.000033,0.246050,0.850758,0.239554,0.093750
4,0.000044,0.194131,0.827372,0.345404,0.291667


In [3]:
trainset, testset = train_test_split(dataset, test_size = 0.1)
trainset, valset = train_test_split(trainset, test_size = 0.1)

In [4]:
trainset.iloc[0]

time                    0.559717
air_temperature_mean    0.525959
pressure                0.198761
wind_direction          0.479109
wind_speed              0.354167
Name: 2138, dtype: float64

In [5]:
train_dataset = WindSpeedDataset(trainset,transform=ComposeTransform([ToTensor()]))
test_dataset = WindSpeedDataset(testset, transform=ComposeTransform([ToTensor()]))
val_dataset = WindSpeedDataset(valset, transform=ComposeTransform([ToTensor()]))

In [6]:
train_dataset[0]

(tensor([0.5260, 0.1988, 0.4791], dtype=torch.float64),
 tensor([0.3542], dtype=torch.float64))

In [7]:
batch_size = 5

trainloader = dataloader.DataLoader(train_dataset, batch_size, shuffle = True)
valloader = dataloader.DataLoader(val_dataset, batch_size, shuffle = False)
testloader = dataloader.DataLoader(test_dataset, batch_size, shuffle = False)

In [8]:
f, l = next(iter(trainloader))

In [9]:
f

tensor([[0.3183, 0.4030, 0.6407],
        [0.4018, 0.5420, 0.3872],
        [0.6953, 0.4560, 0.6880],
        [0.6862, 0.4684, 0.6964],
        [0.1670, 0.6671, 0.3398]], dtype=torch.float64)

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 30)
        self.fc2 = nn.Linear(30, 1)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [11]:
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=3, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=1, bias=True)
)


In [12]:
from torch.optim import SGD
criterion = nn.MSELoss()
optimizer = SGD(params=model.parameters(), lr=0.01)

In [13]:
trainlosses = []
testlosses = []
epochs = 5


for epoch in range(epochs):
    trainloss = 0
    model.train()
    for batch, (data, target) in enumerate(trainloader):
        data = data.type(torch.FloatTensor)
        target = target.type(torch.FloatTensor)
#         if cuda:
#             image, target = image.cuda(), target.cuda()
        
        optimizer.zero_grad()
        
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        trainloss += loss.item()
        
        if batch % 20 == 0:
            testloss = 0
            model.eval()
            with torch.no_grad():
                for data, target in valloader:
                    data = data.type(torch.FloatTensor)
                    target = target.type(torch.FloatTensor)
#                     if cuda:
#                         image, target = image.cuda(), target.cuda()
                    ps = model(data)
                    testloss += criterion(ps, target).item()
                testloss = testloss / len(valloader)        
            trainloss = trainloss / len(trainloader)
        
            trainlosses.append(trainloss)
            testlosses.append(testloss)
        
            print(f'Epoch: {epoch}',
                  f'Batch: {batch} out of {len(trainloader)}',
                  f'Training Loss: {trainloss}',
                  f'Test Loss: {testloss}')

Epoch: 0 Batch: 0 out of 592 Training Loss: 3.357122438280163e-05 Test Loss: 0.036849915219301525
Epoch: 0 Batch: 20 out of 592 Training Loss: 0.0009286321600158176 Test Loss: 0.0241932076610145
Epoch: 0 Batch: 40 out of 592 Training Loss: 0.0006795604045671494 Test Loss: 0.023977849147437762
Epoch: 0 Batch: 60 out of 592 Training Loss: 0.0008818594143051323 Test Loss: 0.023406696215688222
Epoch: 0 Batch: 80 out of 592 Training Loss: 0.0008571129203145197 Test Loss: 0.023168733641575796
Epoch: 0 Batch: 100 out of 592 Training Loss: 0.0008598594183606179 Test Loss: 0.02306930905659542
Epoch: 0 Batch: 120 out of 592 Training Loss: 0.0007085874739950242 Test Loss: 0.022656964818121527
Epoch: 0 Batch: 140 out of 592 Training Loss: 0.0008194323195050002 Test Loss: 0.022549687746284304
Epoch: 0 Batch: 160 out of 592 Training Loss: 0.0008527859742649218 Test Loss: 0.022445912703970505
Epoch: 0 Batch: 180 out of 592 Training Loss: 0.0008271154986396949 Test Loss: 0.0227327468014802
Epoch: 0 Ba

Epoch: 2 Batch: 480 out of 592 Training Loss: 0.0009353106711205243 Test Loss: 0.018220635355012775
Epoch: 2 Batch: 500 out of 592 Training Loss: 0.0005929857606282452 Test Loss: 0.017820934761513137
Epoch: 2 Batch: 520 out of 592 Training Loss: 0.0007770770078566212 Test Loss: 0.017837836382430836
Epoch: 2 Batch: 540 out of 592 Training Loss: 0.000644089535838063 Test Loss: 0.018733670199501583
Epoch: 2 Batch: 560 out of 592 Training Loss: 0.0007339283960409356 Test Loss: 0.01789578387979418
Epoch: 2 Batch: 580 out of 592 Training Loss: 0.0008111443880521478 Test Loss: 0.018289549932391805
Epoch: 3 Batch: 0 out of 592 Training Loss: 1.6128153515023155e-05 Test Loss: 0.0177792228518449
Epoch: 3 Batch: 20 out of 592 Training Loss: 0.0006620145706833872 Test Loss: 0.017848059173433507
Epoch: 3 Batch: 40 out of 592 Training Loss: 0.0007080287736019978 Test Loss: 0.017773602417119862
Epoch: 3 Batch: 60 out of 592 Training Loss: 0.0008015244757640362 Test Loss: 0.01979110015302219
Epoch: 3 

In [16]:
#plt.plot(trainlosses)
plt.plot(testlosses)

In [31]:
result = []
expected = []
for f, l in testloader:
    for i in model(f.type(torch.FloatTensor)).tolist():
        result.append(i[0])
    for j in l.tolist():
        expected.append(j[0])
    #result.append(.item())

In [32]:
plt.plot(result)
plt.plot(expected)